# Bobcat Migration

## Species Description


Bobcats are part of the larger genus of Lynx. There are 4 extant species
of Lynx, one of which is the bobcat. The bobcat is often confused with the
Lynx, but the key difference is that Lynx are larger and their tails are 
all black versus the Bobcat which have white tails with a black strip.

I chose the bobcat *'lynx rufus'* to look at their migration because I 
have a sister who works at the Sonoma County Wildlife Rescue in 
California. She does the [rescue and release](https://scwildliferescue.org/rescue-rehab-release) 
part of the porgram and they frequently rescue injured bobcats, or get in baby 
bobcats that were abandoned in some way that they care for until they 
are old enough to be released. I however, am in Colorado and I have seen
bobcats in the wilderness. This made me curious about their overall migration.

This species's migration ranges between Mexico and Canada, with the 
majority of the bobcats being found in the U.S. They are not a threatened
species, but do suffer from habitat loss. They "live in a wide variety of
habitats, including boreal coniferous and mixed forests in the north, 
bottomland hardwood forests and coastal swamps in the southeast, and 
desert and scrublands in the southwest"(The Smithsonian National Zoo &
Conservation Biology Institute). Another important note that will
inform the later interpretation is that bobcats do not hibernate and are
most active during the winter, particularly January and February which 
is their mating season (The Wildlife Rescue Legaue).

If you are insterested in learning more about bobcats
please visit [The Smithsonian's National Zoo & Conservation Biology 
Institute](https://nationalzoo.si.edu/animals/bobcat) and [The Wildlife
Rescue League](https://www.wildliferescueleague.org/animals/the-bobcat-master-of-the-art-of-concealment/#:~:text=Bobcats%20do%20not%20migrate%2C%20although,large%20as%20six%20square%20miles) .


## Data Description
There were two sources of data used for this project - *Ecoregions* and *GBIF*
data. Ecoregions data is available via a shapefile which can be downloaded
[here](https://www.geographyrealm.com/terrestrial-ecoregions-gis-data/) .
The GBIF data is for species occurances. Together, the two can convey the 
migration spatially and over time of a certain species (if that species' data
is available from GBIF).

* #### Ecoregion Data
Ecoregions are "are areas that are geographically and ecologically similar
and experience similar levels of solar radiation and precipitation". 
There are 846 Ecoregions globally, but most species would be in a small
portion of those edoregions depending on their habitats and migration patterns.
The Ecoregion shapefile used is an updated version from 2017 that was developed
by experts.


**Ecoregion Citation:** 

Terauds, Aleks, et al. “Announcing the Release of Ecoregion Snapshots.” 
    >One Earth, One Earth, 31 May 2024, 
    >www.oneearth.org/announcing-the-release-of-ecoregion-snapshots/. 

* #### GBIF Occurences Data

GBIF stands for [Global Biodiversity Information Facility](https://www.gbif.org/what-is-gbif), 
it "is an international network and data infrastructure funded by the world's 
governments and aimed at providing anyone, anywhere, open access to data 
about all types of life on Earth"(Data Info citation below). This network
draws many data sources together including museum specimans, DNA barcodes,
and crowdsourced photos from smartphones used by non-experts and experts.
GBIF uses data standards to index all these species records.

The data can vary:
1. There may be more occurances recorded in National Parks 
compared to the Arctic, even if the species may similarly present
in different regions. This is because there are fewer people to observe in 
the arctic. 
2. There may be greater or fewer occurances depending on time of year 
that people go outside, how accesible a region is during different times
of year, etc.
3. There may be variation depending on how many people want to provide/upload
data and that also depends on what that person knows or likes more - one
may be more likely to upload species they like or know.

Because this data comes from crowdsourcing and the data will need to be 
normalized which will be further explained in the Methods Description.

**GBIF Citation:**

* Data Info:

    https://www.gbif.org/what-is-gbif


* Data:
 
    GBIF.org (12 October 2024) GBIF Occurrence Download
    https://doi.org/10.15468/dl.sye4x3




## Methods Description 

Because the GBIF data can vary by space and time, the data needs to be 
normalized to account for these issues that would otherwise skew the plot
(ex. make it look like there are more observations in a certain area or time
of year, when that wouldn't be an accurate reflection of reality).
The data was normalized by ecoregion samples, by month samples, and by 
area. This way the normalization is over space and time.

1. Before normalization can happen, the GBIF data was converted into a 
GeoDataFrame using latitude and longitude geometry.

2. Next, a spatial join was performed with parameters (how= 'inner', 
predicate= 'contains'). This identifies the Ecoregion for each observation.

3. Next, observations were grouped by Ecoregion, selecting only month/ecosystem
combinations that have more than one occurance recorded (since a single
occurence could be an error). The .groupby() and .mean() methods were used
to compute the mean occurences by ecoregion and by month.

4. Lastly, divide occurrences by the mean occurrences by month AND the mean
occurrences by ecoregion.

This normalizes the data to be able to have a more accurate plot.

## Bobcat Migration Overtime Plot- While this species does move around, they are not migratory. The map reflects the ecoregions they habitate as well as the time of year they would be more active. 


<embed type="text/html" src="bobcat_migration.html",
width="900" height="900">


The Bobcat Migration Overtime shows a predominance in the U.S. thoughout
most of the year and fewer occurences in Canada and Mexico. There is not 
a striaght forward migration pattern that may be seen in other species
that may 'fly south for the winter' leading to what I have now read in multiple
places is that - bobcats are not migratory. This was not something I researched
prior to delving into the data, but after seeing the plot makes sense. 
The plot also confirms some of the other species information.

June through August tends to have fewer norm occurences in general compared to
the fall and winter months (for the Northern Hemisphere) which have higher
norm occurences. This speaks to the fact that bobcats don't hibernate and 
winter is their mating season, so they would be more likely to be spotted
by a human while looking for a mate. Bobcats are also found around areas
often frequented for skiing/snowboarding and snowshoeing. 

The lack of a straight forward migration pattern speaks to the nature of
bobcats, like the much smaller domestic housecat, are territorial animals.
They tend to occupy the same milage over time, marking it, and only move
under extreme conditions or habitat loss. Because they live in different
types of forests, coastal swamps, and scrubland, they would be threatend
by wildfire and drought. There have been an increase in wildfires in recent
years in the Sierra Nevadas, the Rocky Mountains, and possibly other forest
areas which would threaten or cause them to relocate.


The plot more so speaks to the nature of bobcats and how the data was collected
than it speaks to a distinct migration pattern.

Discussion and conclusion ideally link back to the earlier text for context
(the species description, data, and methods stuff from up top) and interpret
the plot as well